# 使用Claude迭代搜索Wikipedia

[免责声明：此笔记本使用Claude 2模型创建，被认为是遗留版本。]

有些问题无法让Claude信手拈来地直接回答。也许是关于时事的问题。也许您有一个非常详细的问题，而Claude没有记住答案。不用担心！通过一些提示和框架，Claude可以搜索网络来找到答案。在这个笔记本中，我们将创建一个虚拟研究助手，它能够搜索Wikipedia来找到您问题的答案。同样的方法也可以让Claude搜索更广泛的网络，或您提供的一组文档。

这个方法是什么？广义上它属于"工具使用"的范畴。我们创建一个搜索工具，告诉Claude有关它的情况，然后让它开始工作。用伪代码表示：

1. 用搜索工具的描述、最佳使用方式以及如何"调用"它（通过发出特殊字符串）来提示Claude。
2. 告诉Claude您的问题。
3. Claude像平常一样生成token。如果它产生特殊字符串，则终止token生成流，并向搜索API发出查询。
4. 构建一个新提示，由步骤1的提示加上Claude生成到搜索调用字符串之前的所有内容，再加上API调用的结果组成。
5. 重复直到Claude决定完成。

让我们深入了解工具使用和检索的提示。

### 提示词

In [101]:
# Tool Description Prompt
wikipedia_prompt = """You will be asked a question by a human user. You have access to the following tool to help answer the question. <tool_description> Search Engine Tool * The search engine will exclusively search over Wikipedia for pages similar to your query. It returns for each page its title and full page content. Use this tool if you want to get up-to-date and comprehensive information on a topic to help answer queries. Queries should be as atomic as possible -- they only need to address one part of the user's question. For example, if the user's query is "what is the color of a basketball?", your search query should be "basketball". Here's another example: if the user's question is "Who created the first neural network?", your first query should be "neural network". As you can see, these queries are quite short. Think keywords, not phrases. * At any time, you can make a call to the search engine using the following syntax: <search_query>query_word</search_query>. * You'll then get results back in <search_result> tags.</tool_description>"""
print(wikipedia_prompt)

You will be asked a question by a human user. You have access to the following tool to help answer the question. <tool_description> Search Engine Tool * The search engine will exclusively search over Wikipedia for pages similar to your query. It returns for each page its title and full page content. Use this tool if you want to get up-to-date and comprehensive information on a topic to help answer queries. Queries should be as atomic as possible -- they only need to address one part of the user's question. For example, if the user's query is "what is the color of a basketball?", your search query should be "basketball". Here's another example: if the user's question is "Who created the first neural network?", your first query should be "neural network". As you can see, these queries are quite short. Think keywords, not phrases. * At any time, you can make a call to the search engine using the following syntax: <search_query>query_word</search_query>. * You'll then get results back in <

请注意，这个提示中有很多关于如何正确搜索Wikipedia的建议。我们都习惯于只是在Google中输入随机胡言乱语并获得不错的结果，因为查询解析逻辑很好。Wikipedia搜索不是这样的。例如：考虑查询"在阿联酋购买土豆的最佳方式是什么"。在Wikipedia上[这个查询的热门结果](https://en.wikipedia.org/w/index.php?search=What%27s+the+best+way+to+purchase+potatoes+in+the+United+Arab+Emirates&title=Special:Search&profile=advanced&fulltext=1&ns0=1)是关于美国的奴隶制、1973年石油危机、温迪汉堡和Tim Hortons（？？）。而Google正确地带您直接前往Carrefour UAE。

另一个区别是Wikipedia搜索返回整页内容。使用向量搜索，您可能会得到更窄的片段，因此您可能想要更多结果，使用更具体的查询，或两者兼而有之。重点是您的结果在这里的选择上可能会有很大差异，所以要注意！

In [103]:
retrieval_prompt = """Before beginning to research the user's question, first think for a moment inside <scratchpad> tags about what information is necessary for a well-informed answer. If the user's question is complex, you may need to decompose the query into multiple subqueries and execute them individually. Sometimes the search engine will return empty search results, or the search results may not contain the information you need. In such cases, feel free to try again with a different query. 

After each call to the Search Engine Tool, reflect briefly inside <search_quality></search_quality> tags about whether you now have enough information to answer, or whether more information is needed. If you have all the relevant information, write it in <information></information> tags, WITHOUT actually answering the question. Otherwise, issue a new search.

Here is the user's question: <question>{query}</question> Remind yourself to make short queries in your scratchpad as you plan out your strategy."""
print(retrieval_prompt)

Before beginning to research the user's question, first think for a moment inside <scratchpad> tags about what information is necessary for a well-informed answer. If the user's question is complex, you may need to decompose the query into multiple subqueries and execute them individually. Sometimes the search engine will return empty search results, or the search results may not contain the information you need. In such cases, feel free to try again with a different query. 

After each call to the Search Engine Tool, reflect briefly inside <search_quality></search_quality> tags about whether you now have enough information to answer, or whether more information is needed. If you have all the relevant information, write it in <information></information> tags, WITHOUT actually answering the question. Otherwise, issue a new search.

Here is the user's question: <question>{query}</question> Remind yourself to make short queries in your scratchpad as you plan out your strategy.



我们在这里使用scratchpad是出于正常的思维链原因——它让Claude想出一个连贯的计划来回答问题。搜索质量反思用于诱导Claude保持持久性，不要在收集所有相关信息之前就急于回答问题。但为什么要告诉Claude综合信息而不是立即回答呢？

In [104]:
answer_prompt = "Here is a user query: <query>{query}</query>. Here is some relevant information: <information>{information}</information>. Please answer the question using the relevant information."
print(answer_prompt)

Here is a user query: <query>{query}</query>. Here is some relevant information: <information>{information}</information>. Please answer the question using the relevant information.


通过提取信息并在新查询中将其呈现给Claude，我们让Claude将其所有注意力集中在将信息综合为正确答案上。没有这一步，我们发现Claude有时会预先承诺一个答案，然后用搜索结果"证明"它，而不是让结果指导它。

接下来是一大段实现搜索+检索+重新提示的伪代码的代码。

In [88]:
from dataclasses import dataclass
from abc import abstractmethod
import wikipedia
import re
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT
from typing import Tuple, Optional


@dataclass
class SearchResult:
    """
    A single search result.
    """

    content: str


class SearchTool:
    """
    A search tool that can run a query and return a formatted string of search results.
    """

    def __init__():
        pass

    @abstractmethod
    def raw_search(self, query: str, n_search_results_to_use: int) -> list[SearchResult]:
        """
        Runs a query using the searcher, then returns the raw search results without formatting.

        :param query: The query to run.
        :param n_search_results_to_use: The number of results to return.
        """
        raise NotImplementedError()

    @abstractmethod
    def process_raw_search_results(
        self,
        results: list[SearchResult],
    ) -> list[str]:
        """
        Extracts the raw search content from the search results and returns a list of strings that can be passed to Claude.

        :param results: The search results to extract.
        """
        raise NotImplementedError()

    def search_results_to_string(self, extracted: list[str]) -> str:
        """
        Joins and formats the extracted search results as a string.

        :param extracted: The extracted search results to format.
        """
        result = "\n".join(
            [
                f'<item index="{i + 1}">\n<page_content>\n{r}\n</page_content>\n</item>'
                for i, r in enumerate(extracted)
            ]
        )
        return result

    def wrap_search_results(self, extracted: list[str]) -> str:
        """
        Formats the extracted search results as a string, including the <search_results> tags.

        :param extracted: The extracted search results to format.
        """
        return f"\n<search_results>\n{self.search_results_to_string(extracted)}\n</search_results>"

    def search(self, query: str, n_search_results_to_use: int) -> str:
        raw_search_results = self.raw_search(query, n_search_results_to_use)
        processed_search_results = self.process_raw_search_results(raw_search_results)
        displayable_search_results = self.wrap_search_results(processed_search_results)
        return displayable_search_results

In [ ]:
@dataclass
class WikipediaSearchResult(SearchResult):
    title: str


class WikipediaSearchTool(SearchTool):
    def __init__(self, truncate_to_n_tokens: Optional[int] = 5000):
        self.truncate_to_n_tokens = truncate_to_n_tokens
        if truncate_to_n_tokens is not None:
            self.tokenizer = Anthropic().get_tokenizer()

    def raw_search(self, query: str, n_search_results_to_use: int) -> list[WikipediaSearchResult]:
        search_results = self._search(query, n_search_results_to_use)
        return search_results

    def process_raw_search_results(self, results: list[WikipediaSearchResult]) -> list[str]:
        processed_search_results = [
            f"Page Title: {result.title.strip()}\nPage Content:\n{self.truncate_page_content(result.content)}"
            for result in results
        ]
        return processed_search_results

    def truncate_page_content(self, page_content: str) -> str:
        if self.truncate_to_n_tokens is None:
            return page_content.strip()
        else:
            return self.tokenizer.decode(
                self.tokenizer.encode(page_content).ids[: self.truncate_to_n_tokens]
            ).strip()

    def _search(self, query: str, n_search_results_to_use: int) -> list[WikipediaSearchResult]:
        results: list[str] = wikipedia.search(query)
        search_results: list[WikipediaSearchResult] = []
        for result in results:
            if len(search_results) >= n_search_results_to_use:
                break
            try:
                page = wikipedia.page(result)
                print(page.url)
            except Exception:
                # The Wikipedia API is a little flaky, so we just skip over pages that fail to load
                continue
            content = page.content
            title = page.title
            search_results.append(WikipediaSearchResult(content=content, title=title))
        return search_results

In [100]:
def extract_between_tags(tag: str, string: str, strip: bool = True) -> list[str]:
    ext_list = re.findall(f"<{tag}\s?>(.+?)</{tag}\s?>", string, re.DOTALL)
    if strip:
        ext_list = [e.strip() for e in ext_list]
    return ext_list


class ClientWithRetrieval(Anthropic):
    def __init__(self, search_tool: SearchTool, verbose: bool = True, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.search_tool = search_tool
        self.verbose = verbose

    # Helper methods
    def _search_query_stop(
        self, partial_completion: str, n_search_results_to_use: int
    ) -> Tuple[list[SearchResult], str]:
        search_query = extract_between_tags("search_query", partial_completion + "</search_query>")
        if search_query is None:
            raise Exception(
                "Completion with retrieval failed as partial completion returned mismatched <search_query> tags."
            )
        print(f"Running search query against SearchTool: {search_query}")
        search_results = self.search_tool.raw_search(search_query, n_search_results_to_use)
        extracted_search_results = self.search_tool.process_raw_search_results(search_results)
        formatted_search_results = self.search_tool.wrap_search_results(extracted_search_results)
        return search_results, formatted_search_results

    def retrieve(
        self,
        query: str,
        model: str,
        n_search_results_to_use: int = 3,
        stop_sequences: list[str] = [HUMAN_PROMPT],
        max_tokens_to_sample: int = 1000,
        max_searches_to_try: int = 5,
        temperature: float = 1.0,
    ) -> tuple[list[SearchResult], str]:
        prompt = (
            f"{HUMAN_PROMPT} {wikipedia_prompt} {retrieval_prompt.format(query=query)}{AI_PROMPT}"
        )
        starting_prompt = prompt
        print("Starting prompt:", starting_prompt)
        token_budget = max_tokens_to_sample
        all_raw_search_results: list[SearchResult] = []
        for tries in range(max_searches_to_try):
            partial_completion = self.completions.create(
                prompt=prompt,
                stop_sequences=stop_sequences + ["</search_query>"],
                model=model,
                max_tokens_to_sample=token_budget,
                temperature=temperature,
            )
            partial_completion, stop_reason, stop_seq = (
                partial_completion.completion,
                partial_completion.stop_reason,
                partial_completion.stop,
            )
            print(partial_completion)
            token_budget -= self.count_tokens(partial_completion)
            prompt += partial_completion
            if stop_reason == "stop_sequence" and stop_seq == "</search_query>":
                print(f"Attempting search number {tries}.")
                raw_search_results, formatted_search_results = self._search_query_stop(
                    partial_completion, n_search_results_to_use
                )
                prompt += "</search_query>" + formatted_search_results
                all_raw_search_results += raw_search_results
            else:
                break
        final_model_response = prompt[len(starting_prompt) :]
        return all_raw_search_results, final_model_response

    # Main methods
    def completion_with_retrieval(
        self,
        query: str,
        model: str,
        n_search_results_to_use: int = 3,
        stop_sequences: list[str] = [HUMAN_PROMPT],
        max_tokens_to_sample: int = 1000,
        max_searches_to_try: int = 5,
        temperature: float = 1.0,
    ) -> str:
        _, retrieval_response = self.retrieve(
            query,
            model=model,
            n_search_results_to_use=n_search_results_to_use,
            stop_sequences=stop_sequences,
            max_tokens_to_sample=max_tokens_to_sample,
            max_searches_to_try=max_searches_to_try,
            temperature=temperature,
        )
        information = extract_between_tags("information", retrieval_response)[-1]
        prompt = f"{HUMAN_PROMPT} {answer_prompt.format(query=query, information=information)}{AI_PROMPT}"
        print("Summarizing:\n", prompt)
        answer = self.completions.create(
            prompt=prompt, model=model, temperature=temperature, max_tokens_to_sample=1000
        ).completion
        return answer

### 运行查询

我们准备执行查询了！让我们选择一些东西：
- 要比较新的，这样它就不太可能在Claude的训练数据中，
- 且是复合/复杂的问题，因此需要多次搜索。

In [98]:
import os

# Create a searcher
wikipedia_search_tool = WikipediaSearchTool()
ANTHROPIC_SEARCH_MODEL = "claude-2"

client = ClientWithRetrieval(
    api_key=os.environ["ANTHROPIC_API_KEY"], verbose=True, search_tool=wikipedia_search_tool
)

query = "Which movie came out first: Oppenheimer, or Are You There God It's Me Margaret?"

augmented_response = client.completion_with_retrieval(
    query=query,
    model=ANTHROPIC_SEARCH_MODEL,
    n_search_results_to_use=1,
    max_searches_to_try=5,
    max_tokens_to_sample=1000,
    temperature=0,
)
print(augmented_response)

Starting prompt: 

Human: You will be asked a question by a human user. You have access to the following tool to help answer the question. <tool_description> Search Engine Tool * The search engine will exclusively search over Wikipedia for pages similar to your query. It returns for each page its title and full page content. Use this tool if you want to get up-to-date and comprehensive information on a topic to help answer queries. Queries should be as atomic as possible -- they only need to address one part of the user's question. For example, if the user's query is "what is the color of a basketball?", your search query should be "basketball". Here's another example: if the user's question is "Who created the first neural network?", your first query should be "neural network". As you can see, these queries are quite short. Think keywords, not phrases. * At any time, you can make a call to the search engine using the following syntax: <search_query>query_word</search_query>. * You'll 

酷，Claude能够制定计划，执行查询，并将信息综合成准确的答案。注意：如果没有额外的信息提取步骤，Claude有时会正确确定电影的上映日期，但在最终答案中弄错顺序。让我们再试一个。

In [99]:
augmented_response = client.completion_with_retrieval(
    query="Who won the 2023 NBA championship? Who was that team's best player in the year 2009?",
    model=ANTHROPIC_SEARCH_MODEL,
    n_search_results_to_use=1,
    max_searches_to_try=5,
    max_tokens_to_sample=1000,
    temperature=0,
)
print(augmented_response)

Starting prompt: 

Human: You will be asked a question by a human user. You have access to the following tool to help answer the question. <tool_description> Search Engine Tool * The search engine will exclusively search over Wikipedia for pages similar to your query. It returns for each page its title and full page content. Use this tool if you want to get up-to-date and comprehensive information on a topic to help answer queries. Queries should be as atomic as possible -- they only need to address one part of the user's question. For example, if the user's query is "what is the color of a basketball?", your search query should be "basketball". Here's another example: if the user's question is "Who created the first neural network?", your first query should be "neural network". As you can see, these queries are quite short. Think keywords, not phrases. * At any time, you can make a call to the search engine using the following syntax: <search_query>query_word</search_query>. * You'll 

就是这样！您可能注意到搜索工具代码很好且抽象，可以进行微小修改以适应您选择的搜索API。只需记住向Claude解释它需要使用工具的任何提示。您甚至可以给Claude一些理想查询计划和查询结构的少样本示例，以进一步改善性能。